Source: https://github.com/pytorch/examples/tree/master/reinforcement_learning

## REINFORCE

### setup

In [1]:
# import libraries
import argparse
from itertools import count # useful for counting loop

In [2]:
import numpy as np
import gym

In [3]:
# PyTorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable

In [4]:
parser = {
    'gamma': 0.99,  # discount
    'render': False,  # draw or not, False for notebook
    'seed': 7,  # seed for reprocedure
    'log_interval': 10
}
args = argparse.Namespace(**parser)

torch.manual_seed(args.seed)

In [5]:
# init the environment and seed
env = gym.make("CartPole-v0")
env.seed(args.seed)

[2017-04-08 02:55:28,285] Making new env: CartPole-v0


[7]

### model

In [6]:
# define the model
class Policy(nn.Module):
    """State is 4-dimensional vector and there are 2 actions."""
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.affine2 = nn.Linear(128, 2)

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        # return the score of each action (left or right)
        x = F.relu(self.affine1(x))
        action_scores = self.affine2(x)
        return F.softmax(action_scores)

### train

In [7]:
# utilities
def select_action(model, state):
    # in PyTorch, we always use minibatch, so use unsqueeze to make
    # fake batch 1 dimension
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = model(Variable(state))
    action = probs.multinomial()
    model.saved_actions.append(action)
    return action.data

def finish_episode(args, model, optimizer):
    R = 0
    saved_actions = model.saved_actions
    rewards = []
    for r in model.rewards[::-1]:
        R = r + args.gamma * R
        rewards.insert(0, R)
    rewards = torch.Tensor(rewards)
    rewards = (rewards - rewards.mean()) / (
        rewards.std() + np.finfo(np.float32).eps)
    # The following is REINFORCE update, using StochasticVariable node
    # in the paper https://arxiv.org/abs/1506.05254 for more details.
    # For StochasticVariable, to do backward, we need the reward value,
    # so we have to reinforce reward value to the action!
    # See more at http://incompleteideas.net/sutton/williams-92.pdf
    # "REward Increment = Nonnegative Factor x Offset Reinforcement
    # x Characteristic Eligibility,"
    for action, r in zip(model.saved_actions, rewards):
        action.reinforce(r)  # PyTorch implementation for Variable
    optimizer.zero_grad()
    # see http://pytorch.org/docs/autograd.html
    # ?highlight=backward#torch.autograd.backward
    autograd.backward(model.saved_actions,
                      [None for _ in model.saved_actions])
    optimizer.step()  # update parameters
    del model.rewards[:]  # reset to empty list
    del model.saved_actions[:]

In [8]:
def train(args, model, optimizer, running_reward, i_episode):
    state = env.reset()
    for t in range(10000):  # Don't infinite loop while learning
        action = select_action(model, state)
        state, reward, done, _ = env.step(action[0,0])
        if args.render:
            env.render()
        model.rewards.append(reward)
        if done:
            break

    running_reward = running_reward * 0.99 + t * 0.01
    finish_episode(args, model, optimizer)
    if i_episode % args.log_interval == 0:
        print("=> EPISODE {} | Last length: {} | Average length: {:.2f}"
              .format(i_episode, t, running_reward))
    return running_reward, t

### prepare

In [9]:
# init model and define optimizer
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

### run

In [10]:
# training, using running_reward is criterion for stop
running_reward = 10
for i_episode in count(1):
    running_reward, t = train(args, model, optimizer,
                              running_reward, i_episode)
    if running_reward > 200:
        print("Solved! Running reward is now {} and "
              "the last episode runs to {} time steps!"
              .format(running_reward, t))
        break

=> EPISODE 10 | Last length: 14 | Average length: 11.88
=> EPISODE 20 | Last length: 27 | Average length: 12.28
=> EPISODE 30 | Last length: 15 | Average length: 12.81
=> EPISODE 40 | Last length: 38 | Average length: 13.47
=> EPISODE 50 | Last length: 20 | Average length: 15.32
=> EPISODE 60 | Last length: 63 | Average length: 18.94
=> EPISODE 70 | Last length: 58 | Average length: 22.25
=> EPISODE 80 | Last length: 132 | Average length: 26.05
=> EPISODE 90 | Last length: 27 | Average length: 27.45
=> EPISODE 100 | Last length: 23 | Average length: 27.29
=> EPISODE 110 | Last length: 21 | Average length: 27.14
=> EPISODE 120 | Last length: 38 | Average length: 29.51
=> EPISODE 130 | Last length: 82 | Average length: 33.54
=> EPISODE 140 | Last length: 180 | Average length: 40.27
=> EPISODE 150 | Last length: 104 | Average length: 52.51
=> EPISODE 160 | Last length: 24 | Average length: 52.36
=> EPISODE 170 | Last length: 33 | Average length: 50.36
=> EPISODE 180 | Last length: 34 | Av

## Actor-Critic

### setup

In [1]:
import argparse
import gym
from itertools import count
from collections import namedtuple

In [2]:
import numpy as np

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable

In [4]:
parser = {
    "gamma": 0.99,
    "render": False,
    "seed": 7,
    "log_interval": 10
}
args = argparse.Namespace(**parser)

torch.manual_seed(args.seed)

In [5]:
env = gym.make('CartPole-v0')
env.seed(args.seed)

[2017-04-08 02:56:25,417] Making new env: CartPole-v0


[7]

### model

In [6]:
SavedAction = namedtuple('SavedAction', ['action', 'value'])

class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.action_head = nn.Linear(128, 2)
        self.value_head = nn.Linear(128, 1)

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.affine1(x))
        action_scores = self.action_head(x)
        state_values = self.value_head(x)
        return F.softmax(action_scores), state_values

### train

In [7]:
def select_action(model, state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs, state_value = model(Variable(state))
    action = probs.multinomial()
    model.saved_actions.append(SavedAction(action, state_value))
    return action.data

def finish_episode(args, model, optimizer):
    R = 0
    saved_actions = model.saved_actions
    value_loss = Variable(torch.Tensor([0]))
    rewards = []
    for r in model.rewards[::-1]:
        R = r + args.gamma * R
        rewards.insert(0, R)
    rewards = torch.Tensor(rewards)
    rewards = (rewards - rewards.mean()) / rewards.std()
    # indeed, this is still REINFORCE, as in page 273 of Sutton book
    # http://incompleteideas.net/sutton/book/bookdraft2016sep.pdf
    for (action, value), r in zip(saved_actions, rewards):
        reward = r - value.data[0, 0] # Tensor -> numpy
        action.reinforce(reward)
        value_loss += F.smooth_l1_loss(value, Variable(torch.Tensor([r])))
    optimizer.zero_grad()
    final_nodes = [value_loss] + list(map(lambda p: p.action, saved_actions))
    gradients = [torch.ones(1)] + [None] * len(saved_actions)
    autograd.backward(final_nodes, gradients)
    optimizer.step()
    del model.rewards[:]
    del model.saved_actions[:]

In [8]:
def train(args, model, optimizer, running_reward, i_episode):
    state = env.reset()
    for t in range(10000): # Don't infinite loop while learning
        action = select_action(model, state)
        state, reward, done, _ = env.step(action[0,0])
        if args.render:
            env.render()
        model.rewards.append(reward)
        if done:
            break

    running_reward = running_reward * 0.99 + t * 0.01
    finish_episode(args, model, optimizer)
    if i_episode % args.log_interval == 0:
        print("EPISODE {} | Last length: {} | Average length: {:.2f}"
              .format(i_episode, t, running_reward))
    return running_reward, t

### prepare

In [9]:
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)

### run

In [10]:
running_reward = 10
for i_episode in count(1):
    running_reward, t = train(args, model, optimizer,
                              running_reward, i_episode)
    if running_reward > 200:
        print("Solved! Running reward is now {} and "
              "the last episode runs to {} time steps!"
              .format(running_reward, t))
        break

EPISODE 10 | Last length: 10 | Average length: 10.14
EPISODE 20 | Last length: 15 | Average length: 10.18
EPISODE 30 | Last length: 9 | Average length: 10.89
EPISODE 40 | Last length: 36 | Average length: 11.65
EPISODE 50 | Last length: 16 | Average length: 13.09
EPISODE 60 | Last length: 47 | Average length: 15.02
EPISODE 70 | Last length: 157 | Average length: 22.00
EPISODE 80 | Last length: 201 | Average length: 30.53
EPISODE 90 | Last length: 646 | Average length: 47.55
Solved! Running reward is now 228.5757622076188 and the last episode runs to 4846 time steps!
